<a href="https://colab.research.google.com/github/hengoe/DataProcessingInPython_Midterm/blob/main/Midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [98]:
import requests
import time
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
from time import sleep

# Part 1 - API requests

## Exercise 1

In [143]:
url = "https://ies-midterm.soulution.rocks/login/"
params = '{"cuni": "19560968"}'

# use payload of JSON type string ‘{"cuni":"yourCUNInumber"}’ to authenticate
r = requests.post(url = url, data = params)
dataFromLogin = r.json()

# print the personal_code you find in the response,
print("My personal code is: " + dataFromLogin['data']['personal_code'])

# save the list of codes for this is your generated dataset
listOfCodes = dataFromLogin['data']['dataset_ids']

My personal code is: fefc72e8


## Exercise 2

In [ ]:
# for each of the codes you received from /login, send a GET to 
# /data/{data_code} to receive a part of the dataset

In [144]:
lcompanies = []
ldata = []

for data_code in listOfCodes:
  print("Getting data for data_code: " + data_code)
  
  url = "https://ies-midterm.soulution.rocks/data/" + data_code
  params = '{"cuni": "19560968"}'

  # use payload of JSON type string ‘{"cuni":"yourCUNInumber"}’ to authenticate
  r = requests.get(url = url, params = params)
  res = json.loads(r.text)

  # handle error:
  while (not res['data']): # empty dict? try again :)
    r = requests.get(url = url, params = params)
    res = json.loads(r.text)
    print("Got an error while getting the data. Trying again!")

  company = res['data']['company']
  data = pd.DataFrame(res['data']['data'])

  lcompanies.append(company)
  ldata.append(data)

  print("Successfully got data for data_code: " + data_code)
  print("---------------------------------------------------")
  sleep(0.2)

Getting data for data_code: 3e8641892a
Successfully got data for data_code: 3e8641892a
---------------------------------------------------
Getting data for data_code: 8f8c9919a6
Successfully got data for data_code: 8f8c9919a6
---------------------------------------------------
Getting data for data_code: 5df3513c79
Got an error while getting the data. Trying again!
Successfully got data for data_code: 5df3513c79
---------------------------------------------------
Getting data for data_code: 24f88cb68c
Got an error while getting the data. Trying again!
Successfully got data for data_code: 24f88cb68c
---------------------------------------------------
Getting data for data_code: 81395feb08
Successfully got data for data_code: 81395feb08
---------------------------------------------------


# Part 2 - Data analysis


In [122]:
lcompanies

['BHP', 'HIK', 'SLA', 'PSN', 'CCH']

In [145]:
temp = ldata.copy()

In [178]:
l = []
i = 0
for company in lcompanies:
  df = temp[i].set_index('Date')
  df['Company'] = [company]*df.shape[0]
  
  l.append(df)
  i += 1

data = pd.concat(l).reset_index().set_index(['Company','Date'])

In [179]:
data

Open  ...   Volume
Company Date                            ...         
BHP     2019-01-02   46.63999938964844  ...  2003600
        2019-01-03               47.25  ...  3951500
        2019-01-04   48.02000045776367  ...  3996400
        2019-01-07  49.369998931884766  ...  3078700
        2019-01-08   49.70000076293945  ...  2907600
...                                ...  ...      ...
CCH     2020-11-17  18.920000076293945  ...   491872
        2020-11-18  18.950000762939453  ...   756785
        2020-11-19  18.850000381469727  ...   725184
        2020-11-20  19.290000915527344  ...  1020227
                                   NaN  ...      NaN

[1913 rows x 6 columns]

In [203]:
 list(pd.Series(data.index.get_level_values(level=1)).unique())

['2019-01-02',
 '2019-01-03',
 '2019-01-04',
 '2019-01-07',
 '2019-01-08',
 '2019-01-09',
 '2019-01-10',
 '2019-01-11',
 '2019-01-14',
 '2019-01-15',
 '2019-01-16',
 '2019-01-17',
 '2019-01-18',
 '2019-01-22',
 '2019-01-23',
 '2019-01-24',
 '2019-01-25',
 '2019-01-28',
 '2019-01-29',
 '2019-01-30',
 '2019-01-31',
 '2019-02-01',
 '2019-02-04',
 '2019-02-05',
 '2019-02-06',
 '2019-02-07',
 '2019-02-08',
 '2019-02-11',
 '2019-02-12',
 '2019-02-13',
 '2019-02-14',
 '2019-02-15',
 '2019-02-19',
 '2019-02-20',
 '2019-02-21',
 '2019-02-22',
 '2019-02-25',
 '2019-02-26',
 '2019-02-27',
 '2019-02-28',
 '2019-03-01',
 '2019-03-04',
 '2019-03-05',
 '2019-03-06',
 '2019-03-07',
 '2019-03-08',
 '2019-03-11',
 '2019-03-12',
 '2019-03-13',
 '2019-03-14',
 '2019-03-15',
 '2019-03-18',
 '2019-03-19',
 '2019-03-20',
 '2019-03-21',
 '2019-03-22',
 '2019-03-25',
 '2019-03-26',
 '2019-03-27',
 '2019-03-28',
 '2019-03-29',
 '2019-04-01',
 '2019-04-02',
 '2019-04-03',
 '2019-04-04',
 '2019-04-05',
 '2019-04-

In [164]:
colnames = list(data.columns)
colnames

['Open', 'High', 'Low', 'Close', 'Adj.Close', 'Volume', 'Company']

In [204]:
#- the time series should be from 2019-01-02 to 2020-11-20. Is it the case in your sample?
desired = list(pd.Series(data.index.get_level_values(level=1)).unique())
for company in lcompanies:
  if desired == list(data.loc[company].index):
    print("For "+ company + " it is fulfilled!")
  else:
    print("For "+ company + " it is not fulfilled!")

For BHPit is fulfilled!
For HIKit is not fulfilled!
For SLAit is not fulfilled!
For PSNit is not fulfilled!
For CCHit is fulfilled!


In [ ]:
#- what is the highest and lowest Adj Close price the company recorded?
for company in lcompanies:
  temp = 

In [ ]:
#- what day was the highest number of shares traded


In [ ]:
#- what is the total of shares traded


In [ ]:
#- for the company with the highest volume of shares traded over the whole period, plot their share price over time


In [ ]:
#- when did the company record the highest gain and highest loss for the day? (logarithmic loss)


In [ ]:
#- plot the log returns for the company that has the largest difference between opening and closing price. What day and what company did it occur at?


In [ ]:
#- find monthly returns. Which company has the highest recorded?


In [ ]:
#- for stocks with full time series, calculate 10day volatility by calculating the standard
#deviation of the previous 10 days. (i.e. 10th day is the first observation and it includes
#given day)

In [ ]:
#- create a single object with Adjusted Close prices, plot them together in one time series
#plot